In [1]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

# Load dlib's pre-trained model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Define eye aspect ratio calculation function
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Threshold for EAR to consider the eye closed
EAR_THRESHOLD = 0.2

# Load the image
image_path = '/pics/opened/1.jpg'
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = detector(gray, 1)

# Iterate over detected faces
for face in faces:
    landmarks = predictor(gray, face)

    # Get the coordinates of the left and right eye
    left_eye = np.array([(landmarks.part(36).x, landmarks.part(36).y),
                         (landmarks.part(37).x, landmarks.part(37).y),
                         (landmarks.part(38).x, landmarks.part(38).y),
                         (landmarks.part(39).x, landmarks.part(39).y),
                         (landmarks.part(40).x, landmarks.part(40).y),
                         (landmarks.part(41).x, landmarks.part(41).y)], np.int32)

    right_eye = np.array([(landmarks.part(42).x, landmarks.part(42).y),
                          (landmarks.part(43).x, landmarks.part(43).y),
                          (landmarks.part(44).x, landmarks.part(44).y),
                          (landmarks.part(45).x, landmarks.part(45).y),
                          (landmarks.part(46).x, landmarks.part(46).y),
                          (landmarks.part(47).x, landmarks.part(47).y)], np.int32)

    # Calculate EAR for both eyes
    left_ear = eye_aspect_ratio(left_eye)
    right_ear = eye_aspect_ratio(right_eye)

    # Draw the eyes and the result on the image
    cv2.polylines(image, [left_eye], isClosed=True, color=(0, 255, 0), thickness=1)
    cv2.polylines(image, [right_eye], isClosed=True, color=(0, 255, 0), thickness=1)

    # Check if eyes are closed
    if left_ear < EAR_THRESHOLD and right_ear < EAR_THRESHOLD:
        cv2.putText(image, "Eyes Closed", (face.left(), face.top() - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    else:
        cv2.putText(image, "Eyes Open", (face.left(), face.top() - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the result
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'dlib'